In [27]:
//importing spark packages for this task

import org.apache.spark.sql.SparkSession
import spark.implicits._


//create a datafram by reading in the large csv file

val df = spark.read.format("csv").option("header", "true").load("LCLoan.csv")


//show a summary of our target variable

df.groupBy("loan_status").count().show()

+--------------------+-------+
|         loan_status|  count|
+--------------------+-------+
|          Fully Paid|1041952|
|             Default|     31|
|     In Grace Period|   8952|
|Does not meet the...|   1988|
|         Charged Off| 261654|
|            Oct-2015|      1|
|  Late (31-120 days)|  21897|
|             Current| 919695|
|Does not meet the...|    761|
|   Late (16-30 days)|   3737|
+--------------------+-------+



import org.apache.spark.sql.SparkSession
import spark.implicits._
df: org.apache.spark.sql.DataFrame = [id: string, member_id: string ... 143 more fields]


In [28]:
//created a subset of our large dataset
//at 1% of our data, this should result in about 20,000 rows

var subsetDF = df.sample(false, 0.01)



//display a summary of our target variable, 
//and ensure that none of the counts are too out-of-line with the large dataset

subsetDF.groupBy("loan_status").count().show()

+--------------------+-----+
|         loan_status|count|
+--------------------+-----+
|          Fully Paid|10520|
|     In Grace Period|   80|
|Does not meet the...|   27|
|         Charged Off| 2622|
|  Late (31-120 days)|  201|
|             Current| 9015|
|Does not meet the...|    5|
|   Late (16-30 days)|   45|
+--------------------+-----+



subsetDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, member_id: string ... 143 more fields]


In [29]:
//Write a Folder that contains a new csv file. (it will be called part-0000-...)

subsetDF.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("SubsetLCLoan")

In [ ]:
//last,